In [1]:
import bs4 as bs
import urllib.request
import time
from datetime import datetime
import pandas as pd
import json

df = pd.DataFrame()
l=[]
number_of_results = 0

try:

    base_URL = "https://www.immobilienscout24.de/"
    soup = bs.BeautifulSoup(urllib.request.urlopen(base_URL + "Suche/S-1/Haus-Kauf/Nordrhein-Westfalen/Paderborn-Kreis/Paderborn/5,00-/130,00-/EURO--400000,00?enteredFrom=result_list").read(),'lxml')
    for result_count in soup.find_all("span"):
        if r"resultlist-resultCount" in str(result_count.get("data-is24-qa")):
            number_of_results = int(result_count.text)
    
    for seite in range(1, (number_of_results//21) + 2):
    
        print("Loop " + str(seite) + " startet.")
        
        soup = bs.BeautifulSoup(urllib.request.urlopen(base_URL + "Suche/S-"+str(seite)+"/Haus-Kauf/Nordrhein-Westfalen/Paderborn-Kreis/Paderborn/5,00-/130,00-/EURO--400000,00?enteredFrom=result_list"
).read(),'lxml')
        print("Aktuelle Seite: "+"https://www.immobilienscout24.de/Suche/S-2/P-"+str(seite)+"/Haus-Kauf")
        
        #resultlist-resultCount
        for paragraph in soup.find_all("a"):

            if r"/expose/" in str(paragraph.get("href")):
                l.append(paragraph.get("href").split("#")[0])

            l = list(set(l))

        for item in l:

            try:

                soup = bs.BeautifulSoup(urllib.request.urlopen('https://www.immobilienscout24.de'+item).read(),'lxml')

                data = pd.DataFrame(json.loads(str(soup.find_all("script")).split("keyValues = ")[1].split("}")[0]+str("}")),index=[str(datetime.now())])

                data["URL"] = str(item)

                beschreibung = []

                for i in soup.find_all("pre"):
                    beschreibung.append(i.text)

                data["beschreibung"] = str(beschreibung)

                df = df.append(data, sort=False)

            except Exception as e: 
                print(str(datetime.now())+": " + str(e))
                l = list(filter(lambda x: x != item, l))
                print("ID " + str(item) + " entfernt.")
        print("Exportiert CSV")
        df.to_csv("C:/Users/Henning/rohdaten/"+str(datetime.now())[:19].replace(":","").replace(".","")+".csv",
                  sep=";",decimal=",",encoding = "utf-8",index_label="timestamp")     

        print("Loop " + str(seite) + " endet.\n")
        
except Exception as e: 
    print(str(datetime.now())+": " + str(e))

print("FERTIG!")

Loop 1 startet.
Aktuelle Seite: https://www.immobilienscout24.de/Suche/S-2/P-1/Haus-Kauf
Exportiert CSV
Loop 1 endet.

FERTIG!


In [16]:
import os
import pandas as pd
df = pd.DataFrame()
n=0
for i in os.listdir("C:/Users/Henning/rohdaten/"):
    n+=1
    df = df.append(pd.read_csv("C:/Users/Henning/rohdaten/"+str(i),sep=";",decimal=",",encoding="utf-8"), sort=False)
    print("Durchgang "+str(n))
    
df.to_csv("C:/Users/Henning/rohdaten/summery_"+str(datetime.now())[:19].replace(":","").replace(".","")+".csv",
                  sep=";",decimal=",",encoding = "utf-8",index_label="timestamp")   

Durchgang 1
Durchgang 2
